In [11]:
from pathlib import Path 
from geopy.distance import geodesic
import matplotlib.pyplot as plt 
import missingno as msno
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import seaborn as sns
from plotly.subplots import make_subplots

from yellowbrick.regressor import ResidualsPlot
import os
import dask.dataframe as dd #pour importer rapidement la base de données
import sys

In [3]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# Importation de la base

In [4]:
project_root = Path(os.getcwd()).parent  # ou spécifiez directement le chemin
sys.path.append(str(project_root))

# Construire le chemin relatif au projet
file_path = project_root / "bases" / "data2_bon1.csv"

In [5]:
data = pd.read_csv(file_path)

In [65]:
data.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count', 'distance', 'near_airport'],
      dtype='object')

# Nettoyage du dataset

Le nettoyage du dataset consiste en ces principaux points:  


*   Supprimer(filtrer) les valeurs abberrantes(nulles, négatives et trop élévées ) de fare_amount:

le tarif ne doit pas être négatif, il ne doit pas être nul non plus et dans le cadre de New-York, le tarif le plus cher ne dépasse pas 150 $

*  supprimer les valeurs nulles et supérieures à 6 de passenger_count: le nombre de passagers d'un taxi à  NYC ne dépasse pas 6

* Supprimer les latitudes et longitudes hors_norme

In [7]:
def filter(df):
    # Apply all filtering conditions sequentially
    filtered_df = df[
        df["pickup_longitude"].between(-180, 180) &
        df["dropoff_longitude"].between(-180, 180) &
        df["pickup_latitude"].between(-90, 90) &
        df["dropoff_latitude"].between(-90, 90)
    ]
    return filtered_df

# Apply the filter function
data2 = filter(data)
data2= data2[data2.fare_amount > 0]
data2= data2[data2.passenger_count > 0]

il s'agit ici de supprimer les lignes dont la valeur du prix du trajet est supérieur à 400
et les trajets dont le nombre de voyageurs est supérieur à 6

In [8]:
data2=data2[data2['fare_amount']<=400]
data2=data2[data2['passenger_count']<=6]

Nous limiterons nos données dans New York. Nous allons donc exclure les données qui ne sont pas de la ville de New York.

In [9]:
# Limiter les latitudes et longitudes à une zone spécifique (par exemple, New York City)
lat_min = 40  # Latitude minimum de New York
lat_max = 42   # Latitude maximum de New York
lon_min = -75  # Longitude minimum de New York
lon_max = -72  # Longitude maximum de New York

# Filtrer les données pour rester dans ces limites
data2 = data2[
    (data2['pickup_latitude'] >= lat_min) & (data2['pickup_latitude'] <= lat_max) &
    (data2['pickup_longitude'] >= lon_min) & (data2['pickup_longitude'] <= lon_max) &
    (data2['dropoff_latitude'] >= lat_min) & (data2['dropoff_latitude'] <= lat_max) &
    (data2['dropoff_longitude'] >= lon_min) & (data2['dropoff_longitude'] <= lon_max)
]


Création d'une nouvelle variable distance. Il s'agit de la distance entre le point de depart et d'arrivée

In [ ]:
def calculate_distance(row):
    pickup_coords = (row['pickup_latitude'], row['pickup_longitude'])
    dropoff_coords = (row['dropoff_latitude'], row['dropoff_longitude'])

    # Utilisation de geodesic pour calculer la distance en kilomètres
    return geodesic(pickup_coords, dropoff_coords).km

# Appliquer la fonction sur chaque ligne de ton DataFrame
data2['distance'] = data2.apply(calculate_distance, axis=1)


TypeError: only length-1 arrays can be converted to Python scalars

Dans la ville de New York, le transport vers les aréoport comporte des frais suppementaires qu'il est important de noter. De ce faite nous allons 
détecter dans nos données les trajets a destination d'un aréoport ou qui part vers un aréroport. Nous prenons un 1 km comme la distance proche d'un aéroport.

In [ ]:
Airports = [
    [40.641766, -73.780968],  # John F. Kennedy International Airport (JFK)
    [40.776927, -73.873966],  # LaGuardia Airport (LGA)
    [40.689531, -74.174462],  # Newark Liberty International Airport (EWR)
    [40.850935, -74.060858],  # Teterboro Airport (TEB)
    [40.728233, -73.413572],  # Republic Airport (FRG)
    [41.066128, -73.707915],  # Westchester County Airport (HPN)
    [40.753908, -74.007248],  # Manhattan Heliport
    [40.736990, -73.971890],  # Skyports Seaplane Base (SIP)
    [40.738830, -73.969829],  # East 34th Street Heliport
    [41.504093, -74.104839],  # Stewart International Airport (SWF)
    [40.795240, -73.100772],  # MacArthur Airport (ISP)
    [42.748119, -73.802978],  # Albany International Airport (ALB)
]

# Fonction pour calculer la distance entre deux points (Haversine Formula)
def haversine(lon1, lat1, lon2, lat2):
    # Conversion des coordonnées en radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # Calcul des différences
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    # Formule haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Rayon moyen de la Terre en kilomètres
    return c * r
 #Fonction pour vérifier si un point est proche d'un aéroport
def is_near_airport(lon1, lat1, lon2, lat2, threshold=1):
    """
    Vérifie si l'un des points (pickup ou dropoff) est proche
    d'un aéroport (distance < threshold).
    """
    for airport_coords in Airports:
        # Calcul des distances
        distance_to_pickup = haversine(lon1, lat1, airport_coords[1], airport_coords[0])
        if distance_to_pickup < threshold:
            return 1  # Quitte dès qu'un aéroport proche est trouvé
        
        distance_to_dropoff = haversine(lon2, lat2, airport_coords[1], airport_coords[0])
        if distance_to_dropoff < threshold:
            return 1  # Quitte dès qu'un aéroport proche est trouvé
            
    return 0  # 

In [ ]:
def proximity_check(row):
        return is_near_airport(
            row['pickup_longitude'],
            row['pickup_latitude'],
            row['dropoff_longitude'],
            row['dropoff_latitude'],threshold=1
            )

data2['near_airport'] = data2.apply(proximity_check, axis=1)

Des statistique sur la variable qu'on vient de créer

In [ ]:
#les distance negatives ou nulle sont aberantes ,il convient de les exculress.
#Dans la reglementation du transport a New York, le Frais de fransport minimun est de 2.5 dollars en 2015.

In [ ]:
data2=data2[data2["distance"]>0]
data2=data2[data2["fare_amount"]>=2.5]

La variable pick_time conient des informations utile. il sera decomposer a heure, jour, heur.
la variable heure convient d'etre 

In [ ]:
data2['pickup_datetime'] = pd.to_datetime(data2['pickup_datetime'])
data2['hour'] = data2['pickup_datetime'].dt.hour
data2['day_of_week'] = data2['pickup_datetime'].dt.dayofweek  
data2['season'] = data2['pickup_datetime'].dt.month % 12 // 3 + 1
data2['hour_cos'] = np.cos(2 * np.pi * data2['hour'] / 24)

Selection des variables pertinent pour notre modele

In [ ]:
#Separation entre donnee explicative et donnee explique

In [ ]:
y_data = datat[['fare_amount']]
x_data = datat.drop(columns=['fare_amount'])

In [ ]:
Divisions des donnee entre donnee d'entraitenement et donnee de test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_data, test_size=0.2, random_state=42)

# La mise en place des modeles

## Modele de regression simple

### La mise en place du pipeline pour le modele

In [44]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), ['hour_cos','passenger_count', 'distance','pickup_longitude','dropoff_longitude','pickup_latitude','dropoff_latitude']),
        
    ]
)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression()) 
])

### L'entrainement du modele 

In [47]:
pipeline.fit(X_train, y_train)

NameError: name 'X_train' is not defined

### Nous allons donner un nom a notre modele

In [ ]:
model = pipeline.named_steps['model']

## Modele de regression Ridge

### Creation du Pipeline

In [ ]:
pipeline = Pipeline([
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), variables_numeriques)  # Normaliser les variables numériques
        ], 
        remainder='passthrough'  # Laisser les autres colonnes intactes
    )),
    ('ridge', Ridge())  # Modèle Ridge
])

### Les alphas sur lequel chercher le meileur

In [56]:
alpha_values = [18.5,18.4,18,45]

### le mise en place du modele avec 

In [ ]:
param_grid = {'ridge__alpha': alpha_values}  # Utilisation de l'étape 'ridge' du pipeline
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=make_scorer(r2_score))

### Chercher le meilleur apha

In [ ]:
best_alpha = grid_search.best_params_['ridge__alpha']
print(f"Meilleure valeur pour alpha : {best_alpha}")

### Entrainement le modele avec le meileur alpha

In [ ]:
best_model = grid_search.best_estimator_